<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Increase_RAM_Reference_Notes_By_Techhawa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [56]:
import CreatePestImageData_AlexNetCopy as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [11]:
support.Unzip('/content/LABELS_V2.0.zip','/content/Pests/Labels')

In [13]:
support.fileCount('/content/Pests/Labels')

4330


In [57]:
dataSet = Imagedataset.createImageDataset()

In [58]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object , '/content/Pests/Images/',  test_size=11241 )

In [ ]:
listdir('/content/Pests/Labels')

In [45]:
import cv2

img = cv2.imread('/content/Pests/Images/' + '1_2_2_6_1_1_20180824-093745.xml'[0:-4] + '.jpg')

In [ ]:
patt = annot_path +'/' + i
            parsed = self.parse_xml(patt)
            img = cv2.imread(image_Path + i[0:-4] + '.jpg') 

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object , '/content/Pests/Images/',  test_size=11241 )

In [60]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [72]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights=None, include_top=False,input_shape=(48,48,3))
model.layers.append(Dense(64, activation='relu'))
model.layers.append(Dense(10, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.

In [ ]:
import numpy as geek 
  
trainX = geek.load('/content/drive/My Drive/Research-Data/trainX.npy') 
trainY = geek.load('/content/drive/My Drive/Research-Data/trainY') 
testX = geek.load('/content/drive/My Drive/Research-Data/testX') 
testY = geek.load('/content/drive/My Drive/Research-Data/testY') 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [69]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44964, 32, 32, 3)
(44964,)
(11241, 32, 32, 3)
(11241,)


In [ ]:
filepath="/content/drive/My Drive/Models/Inception_V1/weights-{epoch:02d}-{val_output_accuracy:.3f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='val_output_accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

In [ ]:
model.fit(trainX,trainY,verbose=1,epochs=1)